In [1]:
import sys
print(sys.version)
!jupyter --version

3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]
Selected Jupyter core packages...
IPython          : 7.34.0
ipykernel        : 6.29.4
ipywidgets       : 8.1.3
jupyter_client   : 7.4.9
jupyter_core     : 5.7.2
jupyter_server   : 2.14.1
jupyterlab       : 4.2.2
nbclient         : 0.10.0
nbconvert        : 7.16.4
nbformat         : 5.10.4
notebook         : 6.5.6
qtconsole        : 5.5.2
traitlets        : 5.14.3


In [6]:
pip uninstall tensorflow

^C
Note: you may need to restart the kernel to use updated packages.


In [3]:
#pip install tensorflow==2.10.1

Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
print(tf.__version__)

ModuleNotFoundError: No module named 'tensorflow.python.saved_model'

In [2]:
import csv
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding,SimpleRNN,GRU
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from string import punctuation
punct = list(punctuation)
from tqdm import tqdm
from keras.callbacks import ReduceLROnPlateau,EarlyStopping
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

ModuleNotFoundError: No module named 'tensorflow.python.saved_model'

### Data Preprocessing


In [ ]:
data=pd.read_csv('C:\MY FILES\Thesis\Coursera_review\Final notebooks\dataset_small.csv')
data.head()

In [ ]:
# Renaming columns
data = data.rename(columns={'Label':'Rating'})
data = data.rename(columns={'CourseId':'CourseName'})
data.head()

In [ ]:
# Analysing dataframe attributes
print('* Size of dataframe: {}\n'.format(data.shape))
print('* Datatype of columns are:\n {}\n'.format(data.dtypes))
print('* Count of different product categories:\n {}\n'.format(data['Rating'].value_counts()))

In [ ]:
# Plotting word lenghts of reviews
data['Review'] = data['Review'].astype(str)
word_length = [len(x) for x in data['Review']]
plt.plot(word_length)

In [ ]:
# Types of ratings
data['Rating'].value_counts()

### Checking for Class imbalance

In [ ]:
sns.countplot(x=data['Rating'], data=data, hue=data['Rating'], palette='RdBu')
plt.xticks([0,1,2,3,4,5])
plt.show()

In [ ]:
data.head()

### Handling class imbalance problem using imblearn's random over sampling technique

In [ ]:
from imblearn.over_sampling import RandomOverSampler
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='all')
# fit and apply the transform
#X = data.drop(columns=['Rating','CourseName'])
X = data[['Review']]
y = data['Rating']
X_over, y_over = oversample.fit_resample(X, y)
sns.countplot(x=y_over, data=data, hue=y_over, palette='RdBu')
plt.xticks([0,1,2,3,4,5])
plt.show()

In [ ]:
X_over.shape

In [ ]:
y_over.shape

In [ ]:
data.shape

### Combining oversampled columns to create dataframe

In [ ]:
data_over = pd.concat([X_over, y_over], axis=1)

In [ ]:
data_over.shape

In [ ]:
data_over.head()

### Train-Test Splitting

In [ ]:
# Importing train test splilt library
from sklearn.model_selection import train_test_split

# Train-Test Splitting
train_data, test_data = train_test_split(data_over, test_size = 0.20)

In [ ]:
len(train_data),len(test_data)

### Text Preprocessing

In [ ]:
# Importing NLTK Libraries
import nltk
from nltk.corpus import stopwords
from nltk import *

In [ ]:
# Declaring function for text preprocessing

def preprocess_text(main_df):
  df_1 = main_df.copy()

  # remove stopwords
  nltk.download('stopwords')         # Downloading stopwords
  stop = stopwords.words('english')
  df_1['Review'] = df_1['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

  # remove punctuations and convert to lower case
  df_1['Review'] = df_1['Review'].apply(lambda x: re.sub('[!@#$:).;,?&]', '', x.lower()))

  # remove double spaces
  df_1['Review'] = df_1['Review'].apply(lambda x: re.sub(' ', ' ', x))

  return df_1

In [ ]:
# Preprocessing training and test data
train_data = preprocess_text(train_data)
test_data = preprocess_text(test_data)

In [ ]:
# Verifying text preprocessing
train_data['Review'].head()

### Label Encoding

In [ ]:
# Declaring train labels
train_labels = train_data['Rating']
test_labels = test_data['Rating']

In [ ]:
# Converting labels to numerical features
import numpy as np
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train_labels)
train_labels = le.transform(train_labels)
test_labels = le.transform(test_labels)

print(le.classes_)
print(np.unique(train_labels, return_counts=True))
print(np.unique(test_labels, return_counts=True))

In [ ]:
# Changing labels to categorical features
import numpy as np
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
import numpy as np

train_labels = to_categorical(np.asarray(train_labels))
test_labels = to_categorical(np.array(test_labels))

### Tokenizing Sentences and Fixing Sentence Length

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Defining training parameters
max_sequence_length = 200
max_words = 2500

# Tokenizing Reviews/sentences wrt num_words
tokenizer = Tokenizer(num_words = max_words)  # Selects most frequent words
tokenizer.fit_on_texts(train_data.Review)      # Develops internal vocab based on training text
train_sequences = tokenizer.texts_to_sequences(train_data.Review)  # converts text to sequence

test_sequences = tokenizer.texts_to_sequences(test_data.Review)

In [ ]:
# Fixing the sequence length
from tensorflow.keras.preprocessing.sequence import pad_sequences
train_data = pad_sequences(train_sequences, maxlen = max_sequence_length)
test_data = pad_sequences(test_sequences, maxlen = max_sequence_length)
train_data.shape, test_data.shape

### Declaring GRU Model

In [ ]:
embedding_dim = 64

model = Sequential()

model.add(Embedding(max_words,
                   embedding_dim,
                   input_length=max_sequence_length))

model.add(Dropout(0.5))
model.add(GRU(units =32))
model.add(Dropout(0.5))
model.add(Dense(5,activation='softmax'))
model.summary()

### Training GRU Neural Network

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)
early_stop=EarlyStopping(monitor='val_loss',patience=10)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
history = model.fit(train_data, train_labels, epochs=5,
                    validation_data=(test_data, test_labels),
                    callbacks=[reduce_lr,early_stop])

In [ ]:
# Prediction on Test Data
predicted_gru = model.predict(test_data)
predicted_gru

### GRU Model Evaluation

In [ ]:
import sklearn
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(test_labels, predicted_gru.round())

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))
print('################################')
print(sklearn.metrics.classification_report(test_labels, predicted_gru.round()))

### Model Performance with Epochs

In [ ]:
def accuracy_plot(history):

    fig, ax = plt.subplots(1, 2, figsize=(10,4))

    fig.suptitle('Model Performance with Epochs', fontsize = 16)
    # Subplot 1
    ax[0].plot(history.history['accuracy'])
    ax[0].plot(history.history['val_accuracy'])
    ax[0].set_title('Model Accuracy', fontsize = 14)
    ax[0].set_xlabel('Epochs', fontsize = 12)
    ax[0].set_ylabel('Accuracy', fontsize = 12)
    ax[0].legend(['train', 'validation'], loc='best')

    # Subplot 2
    ax[1].plot(history.history['loss'])
    ax[1].plot(history.history['val_loss'])
    ax[1].set_title('Model Loss', fontsize = 14)
    ax[1].set_xlabel('Epochs', fontsize = 12)
    ax[1].set_ylabel('Loss', fontsize = 12)
    ax[1].legend(['train', 'validation'], loc='best')


accuracy_plot(history)

## Implimenting Bi-LSTM Model

## Declaring Model

In [ ]:
# Importing Libraries

import tensorflow as tf
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Conv1D, SimpleRNN
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.layers import Dense, Input, Input, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding

In [ ]:
embedding_dim = 64

# Model Training
model = Sequential()
model.add(Embedding(max_words,
                   embedding_dim,
                   input_length=max_sequence_length))

# Bidirectional LSTM
model.add(Bidirectional(LSTM(16, return_sequences=True, dropout=0.4, recurrent_dropout=0)))

model.add(GlobalMaxPool1D())

model.add(Dense(5,activation='softmax'))

model.summary()


### Training Bi-LSTM

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer='RMSProp', metrics = ['accuracy'])
# training and validating model
history = model.fit(train_data, train_labels, epochs=5,
                    validation_data=(test_data, test_labels),
                    callbacks=[reduce_lr,early_stop])

In [ ]:
# Prediction on Test Data
predicted_lstm = model.predict(test_data)
predicted_lstm

### Bi-LSTM Model Evaluation

In [ ]:
import sklearn
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(test_labels, predicted_lstm.round())

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))
print('################################')
print(sklearn.metrics.classification_report(test_labels, predicted_lstm.round()))

### Model Performance with Epochs

In [ ]:
def accuracy_plot(history):

    fig, ax = plt.subplots(1, 2, figsize=(10,4))

    fig.suptitle('Model Performance with Epochs', fontsize = 16)
    # Subplot 1
    ax[0].plot(history.history['accuracy'])
    ax[0].plot(history.history['val_accuracy'])
    ax[0].set_title('Model Accuracy', fontsize = 14)
    ax[0].set_xlabel('Epochs', fontsize = 12)
    ax[0].set_ylabel('Accuracy', fontsize = 12)
    ax[0].legend(['train', 'validation'], loc='best')

    # Subplot 2
    ax[1].plot(history.history['loss'])
    ax[1].plot(history.history['val_loss'])
    ax[1].set_title('Model Loss', fontsize = 14)
    ax[1].set_xlabel('Epochs', fontsize = 12)
    ax[1].set_ylabel('Loss', fontsize = 12)
    ax[1].legend(['train', 'validation'], loc='best')


accuracy_plot(history)

### Confusion Matrix

In [ ]:
# Declaring function for plotting confusion matrix
import seaborn as sns
from sklearn.metrics import confusion_matrix

def plot_cm(model, test_data, test_labels):

    Rating = ['1', '2', '3', '4', '5']

    # Calculate predictions
    pred = model.predict(test_data)

    # Declaring confusion matrix
    cm = confusion_matrix(np.argmax(np.array(test_labels),axis=1), np.argmax(pred, axis=1))

    # Heat map labels

    group_counts = ['{0:0.0f}'.format(value) for value in cm.flatten()]
    group_percentages = ['{0:.2%}'.format(value) for value in cm.flatten()/np.sum(cm)]

    labels = [f"{v2}\n{v3}" for v2, v3 in zip(group_counts, group_percentages)]
    labels = np.asarray(labels).reshape(5,5)

    # Plotting confusion matrix
    plt.figure(figsize=(8,4))

    sns.heatmap(cm, cmap=plt.cm.Blues, annot=labels, annot_kws={"size": 10}, fmt = '',
                xticklabels = Rating,
                yticklabels = Rating)

    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12, rotation = 'horizontal')
    plt.title('Confusion Matrix\n', fontsize=13)
    plt.xlabel('Predicted Labels', fontsize=10)
    plt.ylabel('Actual Labels', fontsize=10)

plot_cm(model, test_data, test_labels)